In [ ]:
import requests


webhook_url = 'https://discord.com/api/webhooks/1313346849838596106/6Rn_8BNDeL9bMYfFtqscpu4hPah5c2RsNl0rBiPoSw_Qb9RXgDdVHoHmwEuStPv_ufnV'  # 복사한 Discord 웹훅 URL로 변경
username = "Stock Trading Bot"

message = "매수가 완료되었습니다."

data = {
    "content": message,
    "username": username,  # 원하는 이름으로 설정 가능
}

# 요청 보내기
response = requests.post(webhook_url, json=data)

# 응답 확인
if response.status_code == 204:
    print("메시지가 성공적으로 전송되었습니다.")
else:
    print(f"메시지 전송 실패: {response.status_code}, {response.text}")

In [2]:
import os

# 루트 경로 설정 (예: 프로젝트 루트 경로)
project_root = "/Users/hsh/dev_project/sb-fsts"
os.chdir(project_root)  # 루트 경로로 변경

In [2]:
from app.scheduler.auto_trading_scheduler import scheduled_trading

scheduled_trading(id="id1", virtual=False, trading_bot_name="dreaminmindbot", sorting = "")

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


실전투자 API 객체가 성공적으로 생성되었습니다.
dreaminmindbot의 자동 트레이딩을 시작합니다
sorted_result : [StockSymbol(created_at=1745301561544, symbol='358570', symbol_name='지아이이노베이션', type='kosdaq150'), StockSymbol(created_at=1745301588718, symbol='131290', symbol_name='티에스이', type='kosdaq150'), StockSymbol(created_at=1745301571979, symbol='036830', symbol_name='솔브레인홀딩스', type='kosdaq150'), StockSymbol(created_at=1745301542856, symbol='357780', symbol_name='솔브레인', type='kosdaq150'), StockSymbol(created_at=1745301608461, symbol='144510', symbol_name='지씨셀', type='kosdaq150'), StockSymbol(created_at=1745301532106, symbol='035900', symbol_name='JYP Ent.', type='kosdaq150'), StockSymbol(created_at=1745301602379, symbol='114840', symbol_name='아이패밀리에스씨', type='kosdaq150'), StockSymbol(created_at=1745301535540, symbol='257720', symbol_name='실리콘투', type='kosdaq150'), StockSymbol(created_at=1745301529956, symbol='000250', symbol_name='삼천당제약', type='kosdaq150'), StockSymbol(created_at=1745301540653, symbol='005290', symbol_n

/Users/hsh/dev_project/sb-fsts/app/utils/auto_trading_bot.py:2691: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valid = df.iloc[:max_confirmed_idx][df['horizontal_low'].notna()]
/Users/hsh/dev_project/sb-fsts/app/utils/auto_trading_bot.py:2706: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valid = df.iloc[:max_confirmed_idx][df['horizontal_high'].notna()]
/Users/hsh/dev_project/sb-fsts/app/utils/auto_trading_bot.py:2691: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valid = df.iloc[:max_confirmed_idx][df['horizontal_low'].notna()]
/Users/hsh/dev_project/sb-fsts/app/utils/auto_trading_bot.py:2706: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valid = df.iloc[:max_confirmed_idx][df['horizontal_high'].notna()]
/Users/hsh/dev_project/sb-fsts/app/utils/auto_trading_bot.py:2691: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valid = d

💰 시뮬 중: 383310 / 날짜: 2025-07-01 / 사용가능한 예수금: 0
💰 시뮬 중: 393890 / 날짜: 2025-07-01 / 사용가능한 예수금: 0
💰 시뮬 중: 358570 / 날짜: 2025-07-01 / 사용가능한 예수금: 2,332,913


KeyboardInterrupt: 

In [ ]:
from app.utils.crud_sql import SQLExecutor
from app.utils.database import get_db, get_db_session

sql_executor = SQLExecutor()

In [14]:
query = """
        SELECT 
            *
        FROM 
            fsts.test_table
    """

params = {}

with get_db_session() as db:
    result = sql_executor.execute_select(db, query, params)

print(result)

[{'id': 'id1', 'name': 'name1'}, {'id': 'id2', 'name': 'name2'}]


In [13]:
# RETURNING * 꼭 붙여야 함.
query = """
    INSERT INTO fsts.test_table
    (id, name)
    VALUES (:id, :name)
    RETURNING *;
"""

id = 'id2'
name = 'name2'

params = {
    "id": id,
    "name": name
}

with get_db_session() as db:
    result = sql_executor.execute_insert(db, query, params)

Insert succeeded: [{'id': 'id2', 'name': 'name2'}]


In [27]:
from pykis.kis import PyKis, KisAccessToken

kis_id = 'dreaminmind'  # 한국투자증권 HTS ID
app_key = "PSyTGF07QupJyV76XGm3mkgcr4RDvSeODpVZ"
secret_key = "eteoHNN+iHktbHC1TOKNdDc2ecFHqwyA+o1OijESqRtWY2cirhUqbiuFfO5zmEPNqB8/P0RSBuTjZnPq4zc5u3dKHIg/HOFQqmZcCik621aWqti5MBReqNpr/NChcs8edoBKd4cgJaC47m3IKncU4GglKzWNqHtic/4X8lmOAZx0oDGuFkI="
account = "67737279"
# kis_id = 'bnuazz15'
# app_key = "PSSTsRPEqwFOAqiw1WgOsmpsCToF0CT46YjJ"
# secret_key = "oR1B8Z8jdnPb8Xj2m8N5c+A8LVF2Ai/2UCjO6taWq64OvBXf1arxxV6qfZDp49d0zedr90sub1zQqFYMjL76lbev/MKgHoGsrbVXlEYxP6QhyHJ1AIKLSq3sGSvuBcdAp+5PpM9CtSLRWL6SjxKpzZ0hDWlDwGSCWFtM3dL4XfTFIDfRC8Y="
# account = "64156776"

kis = PyKis(
    id=kis_id,             # 한국투자증권 HTS ID
    appkey=app_key,    # 발급받은 App Key
    secretkey=secret_key, # 발급받은 App Secret
    account=account, # 계좌번호 (예: "12345678-01")
    keep_token=True           # 토큰 자동 갱신 여부
)

In [28]:
from pykis import KisBalance

account = kis.account()
balance: KisBalance = account.balance()

In [29]:
non_zero_stocks = [stock for stock in balance.stocks if stock.qty != 0]

balance.stocks = non_zero_stocks

In [30]:
balance

KisIntegrationBalance(
    account_number=KisAccountNumber('67737279-01'),
    deposits={
        'KRW': KisDomesticDeposit(account_number=KisAccountNumber('67737279-01'), currency='KRW', amount=43321, exchange_rate=1)
    },
    stocks=[
        KisDomesticBalanceStock(account_number=KisAccountNumber('67737279-01'), market='KRX', symbol='383310', qty=72, price=25000, amount=1800000, profit=-206400, profit_rate=-10.28708133971291866028708134),
        KisDomesticBalanceStock(account_number=KisAccountNumber('67737279-01'), market='KRX', symbol='393890', qty=134, price=7410, amount=992940, profit=-9380, profit_rate=-0.9358288770053475935828877005)
    ],
    purchase_amount=3008720,
    current_amount=2792940,
    profit=-215780,
    profit_rate=-7.171820574862399957456991678
)

In [33]:
from pykis import KisOrderableAmount

# 엔비디아 주간거래 시장가 매수 가능 금액 조회
orderable_amount: KisOrderableAmount = account.orderable_amount(
    market="KRX",
    price=1,
    symbol="039200"
)

In [36]:
int(orderable_amount.qty)

2336447

In [ ]:
for holding in balance.stocks:
    print(holding)
    symbol = holding.symbol
    original_symbol_name = holding.symbol_name

In [12]:
import boto3
import json

s3_client = boto3.client('s3', region_name='ap-northeast-2', endpoint_url='https://s3.ap-northeast-2.amazonaws.com', config=boto3.session.Config(signature_version='s3v4'))
bucket_name="sb-fsts"

token_save_path = f"credentials/pykis/token.json"

response = s3_client.get_object(Bucket=bucket_name, Key=token_save_path)

# 본문 읽기 및 JSON 파싱
content = response['Body'].read().decode('utf-8')
data = json.loads(content)
data
with open("token.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [13]:
kis = PyKis(
    id=kis_id,             # 한국투자증권 HTS ID
    appkey=app_key,    # 발급받은 App Key
    secretkey=secret_key, # 발급받은 App Secret
    account=account, # 계좌번호 (예: "12345678-01")
    token=KisAccessToken.load("token.json"),
    keep_token=True           # 토큰 자동 갱신 여부
)
kis.token.save("token.json")

KisHTTPError: HTTP 요청에 실패했습니다.
(403) Forbidden
{"error_description":"접근토큰 발급 잠시 후 다시 시도하세요(1분당 1회)","error_code":"EGW00133"}

[  Request  ]: POST https://openapi.koreainvestment.com:9443/oauth2/tokenP
Headers: {'User-Agent': 'PyKis/2.1.3', 'Accept-Encoding': 'gzip, deflate, zstd', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Length': '283', 'Content-Type': 'application/json'}
Params: {}
Body: [PROTECTED BODY]

In [50]:
s3_client.upload_file(
    Filename="token.json",
    Bucket=bucket_name,
    Key=token_save_path
)